In [ ]:
import os
import random
import json
from datetime import datetime
import getpass

# ------------------------- FILES --------------------------
USERS_FILE = "users.json"
SCORES_FILE = "scores.json"

# ------------------------- LOAD DATA ----------------------
def load_users():
    if os.path.exists(USERS_FILE):
        with open(USERS_FILE, 'r') as f:
            return json.load(f)
    return {}

def save_users(users):
    with open(USERS_FILE, 'w') as f:
        json.dump(users, f, indent=4)

def load_scores():
    if os.path.exists(SCORES_FILE):
        with open(SCORES_FILE, 'r') as f:
            return json.load(f)
    return []

def save_scores(scores):
    with open(SCORES_FILE, 'w') as f:
        json.dump(scores, f, indent=4)

# ------------------------- GLOBALS ------------------------
users = load_users()
logged_user = None
is_admin = False

# ------------------------- QUIZ QUESTIONS -----------------
quiz_questions = {
    "DSA": [
        {
            "q": "What is the time complexity of binary search?",
            "options": ["O(n)", "O(log n)", "O(n^2)", "O(1)"],
            "answer": 2
        },
        {
            "q": "Which data structure uses FIFO?",
            "options": ["Stack", "Queue", "Tree", "Graph"],
            "answer": 2
        }
        # Add more questions here
    ],
    "DBMS": [
        {
            "q": "Which SQL command is used to remove a table?",
            "options": ["DELETE", "DROP", "TRUNCATE", "REMOVE"],
            "answer": 2
        },
        {
            "q": "Which key uniquely identifies a record?",
            "options": ["Primary Key", "Foreign Key", "Candidate Key", "Super Key"],
            "answer": 1
        }
    ],
    "PYTHON": [
        {
            "q": "Which keyword is used to define a function in Python?",
            "options": ["func", "def", "function", "define"],
            "answer": 2
        },
        {
            "q": "What is the output of 3 * '5'?",
            "options": ["555", "15", "Error", "None"],
            "answer": 1
        }
    ]
}

# ------------------------- UTILITIES ----------------------
def clear_screen():
    os.system('cls' if os.name == 'nt' else 'clear')

# ------------------------- REGISTRATION ------------------
def register():
    global users
    clear_screen()
    print("\n--- Registration ---")
    while True:
        username = input("Enter username: ").strip()
        if username in users:
            print("User already exists.")
        elif not username:
            print("Name cannot be empty.")
        else:
            break

    email = input("Enter email: ").strip()
    branch = input("Enter branch: ").strip()
    year = input("Enter year: ").strip()
    contact = input("Enter contact number: ").strip()

    while True:
        password = getpass.getpass("Set your password (min 6 chars): ").strip()
        if len(password) < 6:
            print("Password must be at least 6 characters long.")
        else:
            break

    users[username] = {
        "password": password,
        "email": email,
        "branch": branch,
        "year": year,
        "contact": contact,
        "is_admin": False
    }

    save_users(users)
    print(f"\nRegistration successful! Welcome, {username}.")

# ------------------------- LOGIN -------------------------
def login():
    global logged_user, is_admin
    clear_screen()
    print("\n--- Login ---")
    if logged_user:
        print(f"You are already logged in as {logged_user}. Logout first to switch user.")
        return

    username = input("Enter username: ").strip()
    password = getpass.getpass("Enter password: ").strip()

    if username in users and users[username]["password"] == password:
        logged_user = username
        is_admin = users[username].get("is_admin", False)
        print(f"\nWelcome, {username}!")
    else:
        print("\nInvalid credentials.")

# ------------------------- SHOW PROFILE ------------------
def show_profile():
    clear_screen()
    if not logged_user:
        print("Please login first.")
        return
    u = users[logged_user]
    print("\n--- Profile ---")
    print(f"Username: {logged_user}")
    print(f"Email: {u['email']}")
    print(f"Branch: {u['branch']}")
    print(f"Year: {u['year']}")
    print(f"Contact: {u['contact']}")

# ------------------------- UPDATE PROFILE ---------------
def update_profile():
    global users
    clear_screen()
    if not logged_user:
        print("Please login first.")
        return

    print("\n--- Update Profile ---")
    u = users[logged_user]

    email = input(f"Email [{u['email']}]: ").strip()
    branch = input(f"Branch [{u['branch']}]: ").strip()
    year = input(f"Year [{u['year']}]: ").strip()
    contact = input(f"Contact [{u['contact']}]: ").strip()

    if email:
        u['email'] = email
    if branch:
        u['branch'] = branch
    if year:
        u['year'] = year
    if contact:
        u['contact'] = contact

    change_pass = input("Do you want to change password? (y/n): ").lower()
    if change_pass == 'y':
        while True:
            new_pass = getpass.getpass("New password: ").strip()
            if len(new_pass) < 6:
                print("Password must be at least 6 characters.")
            else:
                u['password'] = new_pass
                print("Password updated successfully.")
                break

    users[logged_user] = u
    save_users(users)
    print("Profile updated!")

# ------------------------- LOGOUT ------------------------
def logout():
    global logged_user, is_admin
    clear_screen()
    if logged_user:
        print(f"{logged_user} has been logged out.")
        logged_user = None
        is_admin = False
    else:
        print("No user logged in.")

# ------------------------- QUIZ MODULE -------------------
def attempt_quiz():
    if not logged_user:
        print("Please login first to attempt a quiz.")
        return

    print("\n--- Quiz Categories ---")
    for idx, cat in enumerate(quiz_questions.keys(), 1):
        print(f"{idx}. {cat}")
    choice = input("Select category number: ").strip()
    try:
        cat_list = list(quiz_questions.keys())
        category = cat_list[int(choice)-1]
    except:
        print("Invalid choice.")
        return

    questions = quiz_questions[category][:]
    random.shuffle(questions)
    score = 0
    for i, q in enumerate(questions[:5], 1):  # Attempt 5 random questions
        print(f"\nQ{i}: {q['q']}")
        for idx, opt in enumerate(q['options'], 1):
            print(f"{idx}. {opt}")
        ans = input("Enter option number: ").strip()
        if ans.isdigit() and int(ans) == q['answer']:
            print("Correct!")
            score += 1
        else:
            print(f"Wrong! Correct answer: {q['options'][q['answer']-1]}")

    print(f"\nQuiz completed. Your score: {score}/{len(questions[:5])}")

    # Save score to file
    scores = load_scores()
    scores.append({
        "username": logged_user,
        "category": category,
        "score": score,
        "total": len(questions[:5]),
        "datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    })
    save_scores(scores)

# ------------------------- VIEW SCORES -------------------
def view_scores():
    scores = load_scores()
    clear_screen()
    if not scores:
        print("No scores available.")
        return
    print("\n--- All Scores ---")
    for s in scores:
        print(f"{s['datetime']} | {s['username']} | {s['category']} | {s['score']}/{s['total']}")

# ------------------------- MAIN MENU ---------------------
def main():
    while True:
        print("\n" + "="*50)
        print(" Welcome to LNCT Quiz System ")
        status = f"Logged in as: {logged_user}" if logged_user else "Status: Logged out"
        print(status)
        print("="*50)
        print("""
1. Register
2. Login
3. Show Profile
4. Update Profile
5. Attempt Quiz
6. View Scores (Admin)
7. Logout
8. Exit
""")
        choice = input("Select option (1-8): ").strip()

        if choice == '1':
            register()
        elif choice == '2':
            login()
        elif choice == '3':
            show_profile()
        elif choice == '4':
            update_profile()
        elif choice == '5':
            attempt_quiz()
        elif choice == '6':
            if is_admin:
                view_scores()
            else:
                print("Only admin can view all scores.")
        elif choice == '7':
            logout()
        elif choice == '8':
            clear_screen()
            print("Goodbye!")
            raise SystemExit
        else:
            print("Invalid choice. Please select 1-8.")

if __name__ == "__main__":
    main()



 Welcome to LNCT Quiz System 
Status: Logged out

1. Register
2. Login
3. Show Profile
4. Update Profile
5. Attempt Quiz
6. View Scores (Admin)
7. Logout
8. Exit



Select option (1-8):  1



--- Registration ---


Enter username:  Shubham
Enter email:  kumar@gmail.com
Enter branch:  AIML
Enter year:  2025
Enter contact number:  657584939
Set your password (min 6 chars):  ········



Registration successful! Welcome, Shubham.

 Welcome to LNCT Quiz System 
Status: Logged out

1. Register
2. Login
3. Show Profile
4. Update Profile
5. Attempt Quiz
6. View Scores (Admin)
7. Logout
8. Exit



...

Please login first to attempt a quiz.

 Welcome to LNCT Quiz System 
Status: Logged out

1. Register
2. Login
3. Show Profile
4. Update Profile
5. Attempt Quiz
6. View Scores (Admin)
7. Logout
8. Exit



Select option (1-8):  2



--- Login ---


Enter username:  Shubham
Enter password:  ········



Welcome, Shubham!

 Welcome to LNCT Quiz System 
Logged in as: Shubham

1. Register
2. Login
3. Show Profile
4. Update Profile
5. Attempt Quiz
6. View Scores (Admin)
7. Logout
8. Exit

